In [ ]:
import re
import pandas as pd

In [ ]:
# with open('/home/trishalaswain/arrangeSection.txt') as file:
#     for line in file:
#         print(line.strip())

In [ ]:
with open('/home/trishalaswain/Downloads/test.txt') as file:
    for line in file:
        line = line.strip()
        if ('sentText.' in line or 'previousSec =' in line or 'map.containsKey' in line) and '//' not in line:
            print(line)  

In [ ]:
outer_dict={}

In [ ]:
list_startsWith = []
list_equals = []
list_contains = []
inner_dict = {}
list_1 = []

In [ ]:
def clear():
        list_startsWith.clear()
        list_equals.clear()
        list_contains.clear()

In [ ]:
from collections import defaultdict

In [ ]:
with open('/home/trishalaswain/Downloads/test.txt') as file:
    prevSec = ''
    def_dict = defaultdict(int)
    currentLevel = 0

    for line in file:
        line = line.strip()

        if 'if' in line and 'HPI' in line or 'if' in line and 'history of present illness' in line:
            currentLevel += 1

        if currentLevel == 0: 
            continue

        if 'sentText.' in line and '//' not in line and currentLevel > 0:
            pattern_1 = re.findall('(?:\")(.*)(?:\")', line)
            list_1.append(str(pattern_1)[1:-1])

            and_count = 0
            and_str = ""

            if '&&' in line:
                and_count += 1
                if(and_count > 0):
                    and_str += line
                    
                templine = and_str.split("&&")
                for item in templine:
                    if '' is not item:
                        print(item)
                        temp_pattern = re.findall('(?:\")(.*)(?:\")', item)
                        boundary_pattern = re.findall('.*(startsWith|endsWith|equals|contains).*',item)
                        inner_dict[str(temp_pattern)] = boundary_pattern
            else:
                if '.startsWith' in line:         
                    inner_dict[str(pattern_1)[1:-1]] = 'startsWith'  
                if '.equals' in line:
                    inner_dict[str(pattern_1)[1:-1]] = 'equals'  
                if '.contains' in line:
                    inner_dict[str(pattern_1)[1:-1]] = 'contains'  
                    
        if 'previousSec =' in line and '//' not in line:  
            myset = set(list_1)
            if len(list_1) != len(myset):
                # print("duplicates found in the list")
                seen = set()
                dupes = []
                for x in list_1:
                    if x in seen:
                        dupes.append(x)
                    else:
                        seen.add(x)
                # print('duplicate patterns found -- ',dupes)
            pattern_2 = re.findall('(?:\")(.*)(?:\")', line)
            # prevSec = str(pattern_2)[1:-1]
            # print(prevSec)(sentText.startsWith("orders") && sentText.contains("name")))
            if str(pattern_2)[1:-1] in outer_dict:
                # print('duplicate prevSec found: ',pattern_2)
                old_dict = outer_dict.get(str(pattern_2)[1:-1])
                new_inner_dict = {**inner_dict, **old_dict}
                outer_dict[str(pattern_2)[1:-1]] = new_inner_dict
            else:
                outer_dict[str(pattern_2)[1:-1]] = inner_dict


            # sub_inner_dict = {}
            # for key,value in inner_dict.items():
            #     if '[' in key:
            #         sub_inner_dict[key] = value
            
            outer_dict[str(pattern_2)[1:-1]] = inner_dict
            clear()
            inner_dict = {}
            list_1 = []
            myset.clear()
            

In [ ]:
outer_dict

In [ ]:
# for key,value in outer_dict.items():
#     if 'HPI' in key:
#         for key_in,value_in in value.items():
#             if ('HPI' not in key_in) or ('History of presentIllness' not in key_in):
#                 del value[key_in]


In [ ]:
for out_key, out_value in outer_dict.items():
    # print(out_key)
    for in_key, in_value in out_value.items():
       
        pattern = str(in_key)[1:-1]
        if '[' in in_key:
            pattern = str(in_key)[2:-2]
            
        if 'startsWith' in in_value: 
            secType = 'SectionStartsType'
        if 'equals' in in_value:
            secType = 'SectionType'
        if 'contains' in in_value:
            secType = 'SectionType'
        derivedSectionType = out_key

        print('SectionTypeMap.put(new '+ secType + ' ("' + pattern + '".toLowerCase()), DerivedSectionType.' + derivedSectionType + ")")
        

In [ ]:
pd.DataFrame.from_dict({(i,j): outer_dict[i][j] 
                           for i in outer_dict.keys() 
                           for j in outer_dict[i].keys()},
                       orient='index').reset_index()